In [11]:
import pandas as pd

In [12]:
allratings = pd.read_pickle('/Users/marc/ds/metis/metisgh/beers/allratings.pkl')
beers = pd.read_pickle('/Users/marc/ds/metis/metisgh/beers/beers.pkl')
sims = pd.read_pickle('/Users/marc/ds/metis/metisgh/beers/awsdata/sims.pkl')

In [13]:
# Beer Search
def findBeer(string):
    result = beers[beers.beername.str.contains(string)][['beernum','beername','brewery','avg_review_score']]
    result.columns = ['beernum','beername','brewery','avg_score']
    return result

def getBeernum(index):
    return review_indices[index]

def getIndex(beernum):
    return indices.index(beernum)

def getBeername(beernum):
    return beers[beers.beernum == beernum]

In [14]:
def nearestNeighbors(beernum,df,neighbors):
    distances = []
    for i in df.columns:
        if i != beernum:
            sim = df[i][beernum]
            distances.append((sim, i))
    distances.sort(reverse=True)
    return distances[:neighbors]

In [15]:
def recommendBeers(beernum=None,recommendations=3,neighbors=10,how='overall'):    
    recs = list(set([x[1] for x in nearestNeighbors(beernum,sims,neighbors)]))
    
    result = allratings[allratings.beernum.isin(recs)]
    result = result.groupby(['beernum'],as_index=False)[['beernum','avgscore','look','smell','taste','feel','overall','abv']].mean()
    
    beerinfo = beers[beers.beernum.isin(recs)].drop('abv',axis=1)
    
    output = pd.merge(beerinfo, result,  how='left', left_on=['beernum'], right_on = ['beernum'])
    if how == 'abv':
        output = output[['beernum','beername', 'brewery', 'style1', 'style2', 'style3','abv']]
    else:
        output = output[['beernum','beername', 'brewery', 'style1', 'style2', 'style3','abv'] + [how]]
    
    return output.sort_values(how,ascending=False)[:recommendations]

## Beer Recommender

### Beer Lookup

In [30]:
findBeer('Poker').head(10)

,beernum,beername,brewery,avg_score


In [29]:
recommendBeers(2691,          # Beer Number
               10,              # Number of Recommendations
               100,             # Neighbors (Rank)
               how='overall'   # overall, avgscore, look, smell, taste, feel, abv
               )

,beernum,beername,brewery,style1,style2,style3,abv,overall
97,10817,Sapporo Mugi To Hopppu (trans. Barley And Hop),Sapporo Breweries Ltd.,Lager,Japanese Lagers,Happoshu,0.050,4.125000
95,12766,Tivoli Beer,Tivoli Brewing Company,Lager,German Lagers,Munich Helles Lager,0.051,3.721154
30,12818,Town Lager,Linden Street Brewery,Lager,American Lagers,California Common / Steam Beer,0.045,3.666667
61,9706,Potosi Pilsener,Potosi Brewing Company,Lager,Czech Lagers,Czech Pilsener,0.060,3.663043
94,8972,Old Seattle Lager,Maritime Pacific Brewing Company,Lager,German Lagers,Munich Helles Lager,0.043,3.659091
88,9714,Poznan Zdjecie,Voodoo Brewery,Lager,European Lagers,Euro Strong Lager,0.063,3.625000
1,10348,Rod's Cream Ale,Public House Brewing Company,Ale,American Ales,Cream Ale,0.050,3.536765
51,8443,Narragansett Light,Narragansett Brewing Co.,Lager,American Lagers,Light Lager,0.038,3.510638
96,12242,Suntory Kinmugi Clear Label (金麦クリアラベル),Suntory,Lager,Japanese Lagers,Happoshu,0.050,3.500000
25,12100,Straub American Lager,Straub Brewery,Lager,American Lagers,American Adjunct Lager,0.043,3.459790
